# Example queries on COVID-19 Knowledge Graph

In [1]:
import os
import time
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
NEO4J_HOME = os.getenv('NEO4J_HOME')
print(NEO4J_HOME)

/Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14


Start neo4j database if it is not already running or restart database to load the latest version of the database.

In [4]:
!"$NEO4J_HOME"/bin/neo4j start

Active database: graph.db
Directories in use:
  home:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14
  config:       /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14/conf
  logs:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14/logs
  plugins:      /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14/plugins
  import:       NOT SET
  data:         /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4jDatabases/database-993db298-6374-4f0a-9a9a-d0783480877a/installation-3.5.14/data
  certificates: /Users/peter/Library/Application Support/Neo4j Desktop/Application/neo4

Wait until database is started up

In [5]:
time.sleep(15)

In [6]:
graph = Graph(password="neo4jbinder")

### List coronavirus outbreaks

In [7]:
query = """
MATCH (p:Pathogen)-[:CAUSES]->(o:Outbreak)
RETURN p.acronym as acronym, p.name as pathogen, p.taxonomy_id as taxonomy_id, o.name as outbreak, o.start_date as start_date
"""
graph.run(query).to_data_frame()

acronym                                           pathogen taxonomy_id  \
0  SARS-CoV-2    Severe acute respiratory syndrome coronavirus 2     2697049   
1    MERS-CoV  Middle East respiratory syndrome-related coron...     1335626   
2    SARS-CoV  Severe acute respiratory syndrome-related coro...      694009   

   outbreak start_date  
0  COVID-19       2019  
1      MERS       2012  
2      SARS       2003

### List person demographics and strain information for California

In [8]:
query = """
MATCH (a:Admin1)<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
WHERE a.name = 'California'
RETURN p.age as age, p.sex as sex, p.exposure_location as exposure_location, s.name as strain, s.clade as clade
"""
graph.run(query).to_data_frame()

age     sex exposure_location                strain clade
0     46  Female        California          USA/CA9/2020    A7
1     51  Female        California          USA/CA8/2020  None
2     56  Female             Hubei          USA/CA7/2020    B4
3     44    Male             Hubei          USA/CA6/2020  None
4     54  Female             Hubei          USA/CA5/2020  None
5     57  Female        California          USA/CA4/2020  None
6     57    Male        California          USA/CA3/2020  None
7     54    Male             Hubei          USA/CA2/2020  None
8     38    Male             Hubei          USA/CA1/2020     B
9   None    None        California    USA/CA-PC101P/2020   A2a
10  None    None        California  USA/CA-CDPH-UC4/2020    A7
11  None    None        California  USA/CA-CDPH-UC3/2020    A7
12  None    None        California  USA/CA-CDPH-UC2/2020    A7

#### Same query using parameterized Cypher
Parameters to Cypher queries can be passed as key-value arguments. Parameters in Cypher are named and are wrapped in curly braces.

In [9]:
admin1 = 'California'

query = """
MATCH (a:Admin1{name: {admin1}})<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
RETURN p.age as age, p.sex as sex, p.exposure_location as exposure_location, 
       s.name as strain, s.clade as clade, s.date as date
ORDER BY s.date
"""
graph.run(query, admin1=admin1).to_data_frame().head(100)

age     sex exposure_location                strain clade        date
0     54    Male             Hubei          USA/CA2/2020  None  2020-01-22
1     38    Male             Hubei          USA/CA1/2020     B  2020-01-23
2     44    Male             Hubei          USA/CA6/2020  None  2020-01-27
3     54  Female             Hubei          USA/CA5/2020  None  2020-01-29
4     57  Female        California          USA/CA4/2020  None  2020-01-29
5     57    Male        California          USA/CA3/2020  None  2020-01-29
6     56  Female             Hubei          USA/CA7/2020    B4  2020-02-06
7     51  Female        California          USA/CA8/2020  None  2020-02-10
8     46  Female        California          USA/CA9/2020    A7  2020-02-23
9   None    None        California  USA/CA-CDPH-UC4/2020    A7  2020-02-27
10  None    None        California  USA/CA-CDPH-UC3/2020    A7  2020-02-27
11  None    None        California  USA/CA-CDPH-UC2/2020    A7  2020-02-27
12  None    None        California    USA/CA-PC101P/2020   A2a  2020-03-11

### Where did clade A originate?

In [10]:
clade = 'A'

query = """
MATCH (s:Strain)--(a:Country)
WHERE s.clade STARTS WITH {clade}
RETURN s.clade as clade, s.name, s.date, a.name
ORDER BY s.date
"""
graph.run(query, clade=clade).to_data_frame().head(100)

clade                                 s.name      s.date          a.name
0     A3                 Wuhan/HBCDC-HB-05/2020  2020-01-18  Mainland China
1     A3              Shandong/IVDC-SD-001/2020  2020-01-19  Mainland China
2     A2                   Germany/BavPat1/2020  2020-01-28         Germany
3    A1a                    Italy/INMI1-cs/2020  2020-01-29           Italy
4    A1a                        Italy/SPL1/2020  2020-01-29           Italy
5    A1a                       England/09c/2020  2020-02-09  United Kingdom
6     A3                  Canada/BC_37_0-2/2020  2020-02-16          Canada
7     A3                   Canada/BC_69243/2020  2020-02-20          Canada
8    A2a                        Italy/CDG1/2020  2020-02-20           Italy
9     A7                           USA/CA9/2020  2020-02-23             USA
10   A2a      Netherlands/Berlicum_1363564/2020  2020-02-24     Netherlands
11   A2a                Switzerland/TI9486/2020  2020-02-24     Switzerland
12   A2a                    Brazil/SPBR-01/2020  2020-02-25          Brazil
13   A1a                 England/200940527/2020  2020-02-25  United Kingdom
14   A1a                 England/200960515/2020  2020-02-25  United Kingdom
15   A2a                    Finland/FIN-25/2020  2020-02-25         Finland
16   A2a      Germany/Baden-Wuerttemberg-1/2020  2020-02-25         Germany
17   A2a                 England/200981386/2020  2020-02-26  United Kingdom
18   A1a                 England/200990006/2020  2020-02-26  United Kingdom
19   A2a                      France/N1620/2020  2020-02-26          France
20   A2a                     France/PL1643/2020  2020-02-26          France
21   A2a           NorthernIreland/HSCNI01/2020  2020-02-26  United Kingdom
22   A1a            Switzerland/1000477102/2020  2020-02-26     Switzerland
23   A2a                Switzerland/GE3895/2020  2020-02-26     Switzerland
24    A3                   Canada/BC_13297/2020  2020-02-27          Canada
25    A3                 England/200960041/2020  2020-02-27  United Kingdom
26   A2a                 England/200990660/2020  2020-02-27  United Kingdom
27   A2a                 England/200990723/2020  2020-02-27  United Kingdom
28   A1a               England/20102000106/2020  2020-02-27  United Kingdom
29   A1a                     Georgia/Tb-54/2020  2020-02-27         Georgia
30   A2a              Mexico/CDMX/InDRE_01/2020  2020-02-27          Mexico
31    A6       Netherlands/Tilburg_1363354/2020  2020-02-27     Netherlands
32    A3                     NewZealand/01/2020  2020-02-27     New Zealand
33   A2a                   Nigeria/Lagos01/2020  2020-02-27         Nigeria
34   A2a            Switzerland/1000477377/2020  2020-02-27     Switzerland
35   A2a                Switzerland/AG0361/2020  2020-02-27     Switzerland
36   A2a                Switzerland/BL0902/2020  2020-02-27     Switzerland
37   A2a                Switzerland/GE3121/2020  2020-02-27     Switzerland
38   A2a                Switzerland/GE5373/2020  2020-02-27     Switzerland
39   A2a                Switzerland/GE9586/2020  2020-02-27     Switzerland
40    A7                   USA/CA-CDPH-UC2/2020  2020-02-27             USA
41    A7                   USA/CA-CDPH-UC3/2020  2020-02-27             USA
42    A7                   USA/CA-CDPH-UC4/2020  2020-02-27             USA
43   A1a                       USA/WA3-UW1/2020  2020-02-27             USA
44   A2a                        Wales/PHW1/2020  2020-02-27  United Kingdom
45    A3                   Australia/NSW05/2020  2020-02-28       Australia
46    A3                   Australia/NSW09/2020  2020-02-28       Australia
47   A1a                    Brazil/SPBR-02/2020  2020-02-28          Brazil
48    A3                   Canada/BC_17397/2020  2020-02-28          Canada
49    A3                 England/200990002/2020  2020-02-28  United Kingdom
50   A1a                 England/200990724/2020  2020-02-28  United Kingdom
51   A1a                 England/200990725/2020  20

### Find persons that imported the virus from another location

In [11]:
query = """
MATCH (c:Admin1)<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
WHERE c.name <> p.exposure_location
RETURN c.name as `state/province`, p.age as age, p.sex as sex, p.exposure_location as exposure_location, 
       s.name as strain, s.clade as clade
ORDER BY p.exposure_location
"""
graph.run(query).to_data_frame()

state/province   age     sex     exposure_location  \
0               Kerala    23    Male                 China   
1               Kerala    20  Female                 China   
2          Panama City    40  Female  Comunitat Valenciana   
3     British Columbia    68    Male        Grand Princess   
4            Minnesota  None    None        Grand Princess   
5            Minnesota  None    None        Grand Princess   
6     British Columbia    80  Female             Hong Kong   
7      New South Wales    53    Male                 Hubei   
8      New South Wales    43    Male                 Hubei   
9           Queensland    44    Male                 Hubei   
10            Victoria    50    Male                 Hubei   
11            Flanders    54    Male                 Hubei   
12       Sihanoukville    60    Male                 Hubei   
13             Ontario  None    Male                 Hubei   
14             Ontario    56    Male                 Hubei   
15             England    27  Female                 Hubei   
16             England    23    Male                 Hubei   
17             England    50  Female                 Hubei   
18             Finland    32  Female                 Hubei   
19       Ile de France    47  Female                 Hubei   
20            Zhejiang    32    Male                 Hubei   
21            Zhejiang    32    Male                 Hubei   
22               Lazio    66  Female                 Hubei   
23               Lazio    66  Female                 Hubei   
24           Kathmandu    32    Male                 Hubei   
25          Nonthaburi    74  Female                 Hubei   
26          Nonthaburi    61  Female                 Hubei   
27           Singapore    31  Female                 Hubei   
28           Singapore    66    Male                 Hubei   
29         South Korea  None    None                 Hubei   
30              Taiwan    45    Male                 Hubei   
31              Taiwan    50  Female                 Hubei   
32             Arizona    26    Male                 Hubei   
33          California    56  Female                 Hubei   
34          California    44    Male                 Hubei   
35          California    54  Female                 Hubei   
36          California    54    Male                 Hubei   
37          California    38    Male                 Hubei   
38            Illinois    68    Male                 Hubei   
39            Illinois    63  Female                 Hubei   
40       Massachusetts    21    Male                 Hubei   
41               Texas    50  Female                 Hubei   
42          Washington  None    Male                 Hubei   
43           Wisconsin    52  Female                 Hubei   
44           Thanh Hoa    25  Female                 Hubei   
45     New South Wales    81    Male                  Iran   
46     New South Wales    31    Male                  Iran   
47     New South Wales    38    Male                  Iran   
48     New South Wales    51  Female                  Iran   
49     New South Wales    43    Male                  Iran   
50    British Columbia    43    Male                  Iran   
51    British Columbia    68    Male                  Iran   
52    British Columbia    58  Female                  Iran   
53    British Columbia    58  Female                  Iran   
54    British Columbia    35  Female                  Iran   
55    British Columbia    33  Female                  Iran   
56    British Columbia    57    None                  Iran   
57    British Columbia    54    Male                  Iran   
58    British Columbia    60    Male                  Iran   
59             Finland    29  Female                  Iran   
60             Bavaria    40  Female                  Iran   
61            Auckland  None  Female                  Iran   
62            New York    39  Female                  Iran   
63            Kraainem    47    Male    

### Strains in Sydney

In [12]:
city = 'Sydney'

query = """
MATCH (c:City{name: {city}})<-[:LOCATED_IN]-(p:Person)-[:CARRIES]->(s:Strain)
RETURN c.name as city, s.name as strain, s.clade as clade, p.exposure_location, s.date as date
ORDER BY s.date
"""
graph.run(query, city=city).to_data_frame()

city                strain clade p.exposure_location        date
0   Sydney         Sydney/2/2020  None               Hubei  2020-01-22
1   Sydney  Australia/NSW01/2020     B               Hubei  2020-01-24
2   Sydney         Sydney/3/2020  None     New South Wales  2020-01-25
3   Sydney  Australia/NSW10/2020  None     New South Wales  2020-02-28
4   Sydney  Australia/NSW09/2020    A3     New South Wales  2020-02-28
5   Sydney  Australia/NSW08/2020  None     New South Wales  2020-02-28
6   Sydney  Australia/NSW05/2020    A3                Iran  2020-02-28
7   Sydney  Australia/NSW07/2020    A3     New South Wales  2020-02-29
8   Sydney  Australia/NSW06/2020    A3                Iran  2020-02-29
9   Sydney  Australia/NSW11/2020    A3                Iran  2020-03-02
10  Sydney  Australia/NSW14/2020    A3     New South Wales  2020-03-03
11  Sydney  Australia/NSW13/2020    A3                Iran  2020-03-04
12  Sydney  Australia/NSW12/2020    A3                Iran  2020-03-04